In [ ]:
import re
#PyMuPDF to read PDFs
# import fitz

## Expresion Regulars

In [24]:

def extraer_metados_gaceta(text_pdf):
    """Extract_metadata of a 'gaceta' of the congress on format text"""
    
    metadatos = {}
    
    # ---Date
    date_pattern = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    match_date = re.search(date_pattern,text_pdf)
    if match_date:
        months = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6, "julio": 7, "agosto":8, "septiembre": 9, "octubre": 10, "noviembre": 11, "Diciembre": 12}
        day_month = int(match_date.group(2))
        month_num = months[match_date.group(3).lower()]
        anio = int(match_date.group(4))
        metadatos["fecha"] = f"{day_month:02d}/{month_num:02}/{anio}"
        
    # ---Directos
    pattern_directs = r"(?i)DIRECTORES:\s*(.*?)\s*(?:SECRETARIO|RAMA)"
    match_directs = re.search(pattern_directs, text_pdf)
    if match_directs:
        metadatos["directores"] =  match_directs.group(1).strip()
    
    # -- 3 edicicion de n paginas
    pattern_pages = r"(?i)EDICI[ÓO]N\s+(\d+)\s+P[ÁA]GINAS"
    pattern_pages = re.search(pattern_pages, text_pdf)
    if pattern_pages:
        metadatos["num_paginas"] = int(pattern_pages.group(1))
        
    # -- 4. Year (NUM ROMAN)
    pattern_anio = r"AÑO\s+(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))"
    match_anio = re.search(pattern_anio, text_pdf)
    if match_anio:
        romano = match_anio.group(1)
        metadatos["anio_romano"] = romano
        #conversion from number roman to whole number
        metadatos["anio"] = whole_number(romano)
        
    # ---.5 name principal
    pattern_name = r"(?i)GACETA\s+DEL\s+CONGRESO"
    if re.search(pattern_name, text_pdf):
        metadatos["nombre"] = "Gaceta del congreso"
    
    # -- 8 ISSN 
    pattern_issn = r"(?i)ISSN\s+(\d{4}-\d{3}[\dX])"
    match_issn = re.search(pattern_issn, text_pdf)
    if match_issn:
        metadatos["issn"] = match_issn.group(1)
        
    # ---9 Rama legislativa
    pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+P[ÚU]BLICO"
    if re.search(pattern_branch, text_pdf):
        metadatos["rama"] = "Rama Legislativa del poder publico"
    
    # --- 10 Camara o Senado
    pattern_entity = r"(?i)(C[ÁA]MARA\s+DE\s+REPRESENTANTES|SENADO\s+DE\s+LA\s+REP[ÚU]BLICA)"
    match_entity = re.search(pattern_entity, text_pdf)
    if match_entity:
        metadatos["camara"] = match_entity.group(1)
    
    # --- 11 tipo de documento
    pattern_type = r"^(PONENCIA|ACTA|PROYECTO\s+DE\s+LEY| INFORME|RESOLUCI[ÓO]N|CONCEPTO|PROPOSICI[ÓO]N|CONSTANCIA|OBJECCIONES)(.*)"
    for linea in text_pdf.splitlines():  # Procesa línea por línea
        match_type = re.search(pattern_type, linea, re.IGNORECASE)
        if match_type:
            metadatos["tipo_documento"] = match_type.group(1).strip()
            metadatos["descripcion"] = match_type.group(2).strip()
            break
    return metadatos

In [25]:
def whole_number(romano):
    """Change from number roman to number whole"""
    values = {"M":1000, 'CM': 900, 'D': 500, 'C': 100, 'XC': 90, 'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entire = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in values:
            entire += values[romano[i+i+2]]
            i += 2
        else:
            entire += values[romano[i]]
            i += 1
        return entire

In [ ]:
test_text = """
REPÚBLICA DE COLOMBIA
CONGRESO
DE LA REPÚBLICA
DE COLOMBIA
SENADO DE LA REPÚBLICA
GACETA DEL CONGRESO
AÑO XXXII - N° 17
DIRECTORES:
SENADO Y CÁMARA
(Artículo 36, Ley 5ª de 1992)
IMPRENTA NACIONAL DE COLOMBIA
www.imprenta.gov.co
Bogotá, D. C., jueves, 9 de febrero de 2023
GREGORIO ELJACH PACHECO
SECRETARIO GENERAL DEL SENADO
www.secretariasenado.gov.co
ISSN 0123 - 9066
EDICIÓN DE 22 PÁGINAS
JAIME LUIS LACOUTURE PEÑALOZA
SECRETARIO GENERAL DE LA CÁMARA
www.camara.gov.co
RAMA LEGISLATIVA DEL PODER PÚBLICO
SENADO DE LA REPÚBLICA
CONCEPTOS JURÍDICOS
CONCEPTO JURÍDICO DEL  MINISTERIO DE SALUD Y PROTECCIÓN SOCIAL SOBRE...
"""

In [ ]:
metadatos = extraer_metados_gaceta(test_text)
print(metadatos) 

{'fecha': '09/02/2023', 'anio_romano': 'XXXII', 'anio': 10, 'nombre': 'Gaceta del congreso', 'rama': 'Rama Legislativa del poder publico', 'camara': 'SENADO DE LA REPÚBLICA', 'tipo_documento': 'CONCEPTO', 'descripcion': 'S JURÍDICOS'}
